In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from statistics import mean
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
pd.set_option('display.float_format', '{:.1f}'.format)

In [2]:
%cd /Users/macbookpro/Documents/COMP90042_NLP/Project/COMP90042_2024_Project

/Users/macbookpro/Documents/COMP90042_NLP/Project/COMP90042_2024_Project


In [3]:
with open("data/train-claims.json", 'r') as file:
    train_claims = json.load(file)

with open("data/dev-claims.json", 'r') as file:
    dev_claims = json.load(file)

with open("data/test-claims-unlabelled.json", 'r') as file:
    test_claims_unlabelled = json.load(file)

with open("data/dev-claims-baseline.json", 'r') as file:
    dev_claims_baseline = json.load(file)
    
with open("data/evidence.json", 'r') as file:
    evidence = json.load(file)

with open("data/external/evidence_climate_2.json", 'r') as file:
    evidence_climate_2 = json.load(file)
    
label_tags = ["SUPPORTS", "REFUTES", "NOT_ENOUGH_INFO", "DISPUTED"]

In [8]:
train_claims_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"],
                    "claim_label": value["claim_label"],
                    "evidences_id": value["evidences"]} for (key,value) in train_claims.items()]
train_claims_df = pd.json_normalize(train_claims_dic)

dev_claims_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"],
                    "claim_label": value["claim_label"],
                    "evidences_id": value["evidences"]} for (key,value) in dev_claims.items()]
dev_claims_df = pd.json_normalize(dev_claims_dic)

dev_claims_baseline_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"],
                    "claim_label": value["claim_label"],
                    "evidences_id": value["evidences"]} for (key,value) in dev_claims_baseline.items()]
dev_claims_baseline_df = pd.json_normalize(dev_claims_baseline_dic)

test_claims_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"]} for (key,value) in test_claims_unlabelled.items()]
test_claims_df = pd.json_normalize(test_claims_dic)

evidence_dic = [{"evidence_id": key,
                             "evidence_text": value
                             } for (key, value) in evidence.items()]
evidence_df = pd.json_normalize(evidence_dic)

Prepare data

In [6]:
from nltk.tokenize import word_tokenize
evid_climate = [word_tokenize(text.lower()) for text in evidence_climate_2.values()]

In [11]:
train_claim_text = [word_tokenize(text.lower()) for text in train_claims_df['claim_text']]
dev_claim_text = [word_tokenize(text.lower()) for text in dev_claims_df['claim_text']]
test_claim_text = [word_tokenize(text.lower()) for text in test_claims_df['claim_text']]

In [12]:
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# Creating a BM25 object
bm25 = BM25Okapi(evid_climate+train_claim_text)

In [29]:
import numpy as np
train_claim_embd = [bm25.get_scores(text) for text in train_claim_text]
train_claim_embd = np.array(train_claim_embd)

In [30]:
dev_claim_embd = [bm25.get_scores(text) for text in dev_claim_text]
dev_claim_embd = np.array(dev_claim_embd)
print("dev claim embedding shape", dev_claim_embd.shape)

test_claim_embd = [bm25.get_scores(text) for text in test_claim_text]
test_claim_embd = np.array(test_claim_embd)
print("test claim embedding shape", test_claim_embd.shape)

evd_embd = [bm25.get_scores(text) for text in evid_climate]
evd_embd = np.array(evd_embd)
print("evidence embedding shape", evd_embd.shape)

dev claim embedding shape (154, 522731)
test claim embedding shape (153, 522731)


: 

In [ ]:
with open("data/BM25/train_claim_embd.json", "w") as file:
    json.dump(train_claim_embd, file)

with open("data/BM25/dev_claim_embd.json", "w") as file:
    json.dump(dev_claim_embd, file)

with open("data/BM25/test_claim_embd.json", "w") as file:
    json.dump(test_claim_embd, file)

with open("data/BM25/evd_embd.json", "w") as file:
    json.dump(evd_embd, file)

In [17]:


claim_scores = bm25.get_scores(train_claim_text[0])
evid_score = bm25.get_scores(evid_climate[0])

In [27]:
evid_score.reshape(-1,1).shape

(522731, 1)

In [23]:
claim_scores.reshape(1,-1).shape

(1, 522731)

In [28]:
cosine_similarity(claim_scores.reshape(1,-1), evid_score.reshape(1,-1))

array([[0.89884075]])